In [1]:
#covariance shift correction
%load_ext autoreload
%autoreload 2
from __future__ import print_function
import os,sys
sys.path.append('../')

## Math and dataFrame
import numpy as np
import pandas as pd
import scipy
import seaborn as sns

import matplotlib.pyplot as plt

#ML
from sklearn.model_selection import cross_val_predict, StratifiedKFold, KFold
from sklearn.metrics import log_loss, accuracy_score, f1_score, confusion_matrix

label_cols = ['toxic', 'severe_toxic', 'obscene',  'threat', 'insult', 'identity_hate']

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

train['dirtyness'] = train.apply(lambda x: x.iloc[2::].sum(), axis = 1)
test['dirtyness'] = test.apply(lambda x: x.iloc[2::].sum(), axis = 1)

COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

print("train set len ", len(train) )
print("test set len ", len(test) )
print("clean samples", len(train[train['dirtyness'] == 0]))
print("toxic samples", len(train[train['dirtyness'] != 0]))

x=train.iloc[:,2:].sum()
#plot
plt.figure(figsize=(8,4))
ax= sns.barplot(x.index, x.values, alpha=0.8)

display(train[:10])

/Users/saizhang/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


train set len  159571
test set len  153164
clean samples 143346
toxic samples 16225


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,dirtyness
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,4
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,0


In [8]:
def evaluate_features(X, y, clf=None):
    """General helper function for evaluating effectiveness of passed features in ML model
    
    Prints out Log loss, accuracy, and confusion matrix with 3-fold stratified cross-validation
    
    Args:
        X (array-like): Features array. Shape (n_samples, n_features)
        
        y (array-like): Labels array. Shape (n_samples,)
        
        clf: Classifier to use. If None, default Log reg is use.
    """
    if clf is None:
        raise ValueError("clf NOne")
    
    probas = cross_val_predict(clf, X, y, cv=KFold(random_state=8), 
                              n_jobs=-1, method='predict_proba', verbose=2)
    pred_indices = np.argmax(probas, axis=1)
    classes = np.unique(y)
    preds = classes[pred_indices]
    print('Log loss: {}'.format(log_loss(y, probas)))
    print('Accuracy: {}'.format(accuracy_score(y, preds)))
    print('F1 score: {}'.format(f1_score(y, preds)))
    print( confusion_matrix(y, preds) )
    

In [3]:
from models.FeatureExtraction import FeatureExtraction
fe = FeatureExtraction()

term_doc = fe.tfIdf(pd.concat([train, test]), 'comment_text')

In [33]:
# use logistic regression to predict whether sample x comes from test disribution or train distribution
from sklearn.linear_model import LogisticRegression

trn_term_doc = term_doc.tocsr()[0:len(train), :]
test_term_doc = term_doc.tocsr()[len(train)::, :]
#print ("feature shape = {}".format(trn_term_doc.shape))
trn_label = np.zeros(len(train)).reshape(-1,1)
test_label = np.ones(len(test)).reshape(-1,1)

term_doc = scipy.sparse.vstack([trn_term_doc, test_term_doc])
label = np.vstack([trn_label, test_label]).reshape(1,-1)[0]
assert(len(label) == term_doc.shape[0])

#shuffle
from sklearn.utils import shuffle
term_doc_re, label_re= shuffle(term_doc, label, random_state=0)

clf = LogisticRegression(penalty="l2", dual=True, 
           tol=0.0001, C=1, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=32, 
           max_iter=200, verbose=0, 
           warm_start=False, n_jobs=1)

evaluate_features(term_doc_re, label_re, clf)
clf.fit(term_doc_re, label_re)
trn_proba = clf.predict_proba(trn_term_doc)
#trn_weights = np.exp(trn_proba)
trn_weights = np.array(zip(*trn_proba)[1]) / np.array(zip(*trn_proba)[0])
trn_weights = np.clip(trn_weights, 1e-12, 200)

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   11.2s finished


Log loss: 0.41458771726
Accuracy: 0.823543894991
F1 score: 0.804483999887
[[144019  15552]
 [ 39632 113532]]


In [34]:

df = pd.DataFrame({'weights': trn_weights})
df.describe()

,weights
count,1.595710e+05
mean,9.969856e-01
std,9.277806e+00
min,2.806437e-08
25%,1.483099e-01
50%,2.666795e-01
75%,4.358300e-01
max,2.000000e+02
